<a href="https://colab.research.google.com/github/raghav7442/Finetuning-Qlora-Huggingface/blob/main/testing_finetuningModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "suzall/Llama-2-7b-chat-finetune-link-box"
prompt = "What is a large language model?"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)


sequences = pipeline(
    f'[INST] {prompt} [/INST]',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Result: [INST] What is a large language model? [/INST]  A large language model is a type of artificial intelligence (AI) model that is trained on a large corpus of text data to generate language outputs that are coherent and natural-sounding. The model is designed to learn the patterns and structures of language, such as grammar, syntax, and semantics, by analyzing the large dataset of text.

The size of a large language model can vary, but it is typically measured in terms of the number of parameters or the amount of training data used to train the model. Some common types of large language models include:

1. Transformer models: These are a type of neural network architecture that are particularly well-suited for natural language processing tasks, such as language translation and language generation. Transformer models typically have millions of parameters and are trained on large datasets of text.
2. Recurrent neural networks (RNNs): These are


In [ ]:
prompt="summarise this : Patient is a 45 year-old Male who presents with complaint of heartburn. The heartburn is located in the stomach, and radiates to the throat. The discomfort began about 2 days ago. The heartburn occurs after exercise, at a restaurant, while lying down and mainly happens in the daytime. It has been constant and worsening. The severity of the heartburn is described as severe. The patient believes it may be due to stress. avoiding fatty foods makes it better. Other symptoms associated with the heartburn include cough. The patient recalls specific incidents where the heartburn worsened, such as after exercise."


In [ ]:

prompt_template = (
    "<s>[INST]\n"
    "<<SYS>>\n"
    "{system_prompt}\n"
    "<</SYS>>\n\n"
    "{user_prompt}"
    "[/INST]"
)
system_prompt = """
    You are a professional medical assistant tasked with improving the accuracy and readability of symptom summaries.
    You will receive a user-provided summary containing grammatical mistakes, vague details, and inaccuracies.
    Your job is to recreate a concise, precise, and medically accurate summary.
    the output will be more detiled than the user summary, as we have mentioned below..

    For example:
    User Summary: Patient is a 42 year-old Male who presents with complaint of heartburn. The heartburn is located in the chest, and the throat. The heartburn is located in yes.
    AI Summary: The patient is a 42-year-old male presenting with complaints of heartburn located in the chest and throat. Symptoms began 3 months ago, worsening over time...

    You will tailor your summary to each input provided by the user.
    """

user_prompt = "What is a large language model?"

prompt = prompt_template.format(system_prompt=system_prompt, user_prompt=user_prompt)


sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
    return_full_text=False,
    temperature=0.5,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

NameError: name 'pipeline' is not defined

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask, render_template, request, jsonify
from transformers import AutoTokenizer

import transformers
import torch
import json
from datetime import datetime

app = Flask(__name__)
# run_with_ngrok(app)

# Initialize model and tokenizer
model_name = "suzall/Llama-2-7b-chat-finetune-link-box"
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Define system prompt
SYSTEM_PROMPT = """You are a helpful AI assistant specialized in generating concise and accurate summaries.
Your task is to:
1. Understand the user's input text thoroughly
2. Generate a clear, well-structured summary
3. Maintain the key points while being concise
4. Use professional and objective language
5. Ensure the summary is coherent and logically organized

Please provide your summary in a clear format."""

def generate_ai_summary(user_input):
    """Generate summary using the LLM model with system prompt"""
    # Combine system prompt and user input in Llama2 chat format
    formatted_prompt = f'[INST] <<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>>\n\n{user_input} [/INST]'

    sequences = pipeline(
        formatted_prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,
    )
    return sequences[0]['generated_text']

def save_to_jsonl(data):
    """Save the data to a JSONL file"""
    filename = 'summaries.jsonl'
    with open(filename, 'a') as f:
        json_record = {
            'timestamp': datetime.now().isoformat(),
            'user_summary': data['user_summary'],
            'ai_summary': data['ai_summary'],
            'rating': data['rating']
        }
        f.write(json.dumps(json_record) + '\n')

# Route for the home page
@app.route('/', methods=['GET'])
def home():
    return render_template('index.html')

# Route for handling the summary generation and rating
@app.route('/generate', methods=['POST'])
def generate_summary():
    user_summary = request.json.get('user_summary')

    # Generate AI summary
    ai_summary = generate_ai_summary(user_summary)

    return jsonify({'ai_summary': ai_summary})

# Route for saving the final summary with rating
@app.route('/save', methods=['POST'])
def save_summary():
    data = {
        'user_summary': request.json.get('user_summary'),
        'ai_summary': request.json.get('ai_summary'),
        'rating': float(request.json.get('rating', 5.0))  # Default rating is 5.0
    }

    save_to_jsonl(data)
    return jsonify({'status': 'success'})

if __name__ == '__main__':
    app.run()

ValueError: Could not load model suzall/Llama-2-7b-chat-finetune-link-box with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py", line 289, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 4264, in from_pretrained
    ) = cls._load_pretrained_model(
  File "/usr/local/lib/python3.10/dist-packages/transformers/model